# Use of a Neural-Network for Constitutive Law

Import all the useful libraries before first run
We need here the classic ones such as:
- math
- numpy
- pandas
- matplotlib

And for the Neural Network, we also need to import parts of the keras module of TensorFlow

In [1]:
import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import exp,log

## Load the test database

Read the Test database

In [2]:
dataPath = '.'
read = np.load(dataPath + '/DatatestWithDerivatives.npz')
testData = read['testData']
nrows = testData.shape[0]

In [3]:
eps_test = testData[:,0].reshape(nrows,1)
epsp_test = testData[:,1].reshape(nrows,1)
T_test = testData[:,2].reshape(nrows,1)
sig_test = testData[:,3].reshape(nrows,1)

 ## Load the NN parameters

In [4]:
ANN_name = '3-15-7-1-sigmoid'

NN = np.load(dataPath + '/' + ANN_name + '.npz')
print (NN.files)
w1 = NN['w1']
w2 = NN['w2']
w3 = NN['w3']
b1 = NN['b1']
b2 = NN['b2']
b3 = NN['b3']
minEntries = NN['minEntries']
maxEntries = NN['maxEntries']
rangeEntries = maxEntries - minEntries
logBase = NN['logBase']
w1, b1, w2, b2, w3, b3, minEntries, maxEntries, rangeEntries, logBase

['logBase', 'minEntries', 'maxEntries', 'w1', 'b1', 'w2', 'b2', 'w3', 'b3']


(array([[-1.71193019e-01, -4.98235881e-01,  1.57230926e+00],
        [-1.82183892e-01, -2.79642552e-01,  1.38154745e+00],
        [-1.21008851e-01, -9.36221778e-02, -1.28485608e+00],
        [ 6.77952528e-01,  7.85129726e-01, -3.60040736e+00],
        [-8.22493362e+00,  1.02857426e-01, -1.17000186e+00],
        [-2.62212038e-01, -3.70604903e-01,  1.53059375e+00],
        [ 3.63313168e-01,  3.07944298e-01, -1.73980820e+00],
        [-1.71282619e-01, -9.36020970e-01, -7.00896144e-01],
        [-2.93405890e+00,  2.49693431e-02, -8.18501532e-01],
        [-2.98778355e-01, -5.17931700e-01, -7.58085251e+00],
        [ 1.17702794e+00, -3.08994055e-01, -1.68417966e+00],
        [-2.31339216e-01, -2.90423155e-01,  1.50245142e+00],
        [ 2.35784103e+02, -9.92930010e-02,  7.60096788e-01],
        [-2.33444443e+01,  1.64729804e-01, -9.49119210e-01],
        [ 3.29279006e-01,  5.42672016e-02, -1.55521262e+00]], dtype=float32),
 array([[-0.5497108 ],
        [-1.0076679 ],
        [ 1.146429  ],

## Reshape data

In [5]:
epsp_test = np.log(epsp_test / logBase)
inputTest = (np.hstack([eps_test, epsp_test, T_test]) - np.array(minEntries)[0:3]) / np.array(rangeEntries)[0:3]
inputTest

array([[0.64999785, 0.90552119, 0.65628871],
       [0.77199114, 0.8999689 , 0.45541125],
       [0.64433951, 0.99445344, 0.11009941],
       ...,
       [0.75192854, 0.99520977, 0.75841439],
       [0.74276917, 0.88747863, 0.12681504],
       [0.68998248, 0.88707664, 0.5388862 ]])

Computes the target

In [6]:
def tanh2Layers(inputData):
    l1 = w1.dot(inputData) + b1
    f1 = np.tanh(l1)
    l2 = w2.dot(f1) + b2
    f2 = np.tanh(l2)
    sigP = w3.dot(f2) + b3
    SigmaNN = sigP * rangeEntries[3] + minEntries[3]
    return SigmaNN

In [7]:
def sigmoid2Layers(inputData):
    l1 = w1.dot(inputData) + b1
    f1 = 1/(1 + np.exp(-l1))
    l2 = w2.dot(f1) + b2
    f2 = 1/( 1 +np.exp(-l2))
    sigP = w3.dot(f2) + b3
    SigmaNN = sigP * rangeEntries[3] + minEntries[3]
    return SigmaNN

Setup data

In [8]:
inp = inputTest.T
sig = sig_test
#inp, sig

Rebuild the real $\sigma$

In [9]:
if ('tanh' in ANN_name) : SigmaNN = tanh2Layers(inp)
elif ('sigmoid' in ANN_name) : SigmaNN = sigmoid2Layers(inp)
else : SigmaNN = 0
SigmaNN

array([[1232.2282292 , 1335.3603057 , 1470.46979899, ..., 1208.21439896,
        1465.07933384, 1287.15287545]])

In [10]:
w1.shape, inp.shape, b1.shape

((15, 3), (3, 5000), (15, 1))

In [12]:
var = "%.12f"
comment = 'C '
cline = 'C **********************************************************************\n'
block =   '      '

def splitFortranLine(line):
    line=line.replace('| +',' +\n     + ')
    line=line.replace('| -',' -\n     + ')
    return line

def toFortran(line):
    line = line.replace("--", "+")
    line = line.replace("+-", "-")
    line = line.replace("(+", "(")
    line = line.replace('+',' + ')
    line = line.replace('-',' - ')
    line = line.replace('=  - ','=-')
    line = line.replace('( - ','(-')
    line = line.replace('  +  ',' + ')
    line = line.replace('.000000000000','.0')
    line = block + line + '\n'
    line = splitFortranLine(line)
    return line


In [86]:
code = ''
code += cline
code += comment + 'Function to compute the ANN : ' + ANN_name + ' yield stress\n'
code += cline

code += '      subroutine vuhard (\n'
code += 'C Read only -\n'
code += '     1     nblock,\n'
code += '     2     nElement, nIntPt, nLayer, nSecPt,\n'
code += '     3     lAnneal, stepTime, totalTime, dt, cmname,\n'
code += '     4     nstatev, nfieldv, nprops,\n'
code += '     5     props, tempOld, tempNew, fieldOld, fieldNew,\n'
code += '     6     stateOld,\n'
code += '     7     eqps, eqpsRate,\n'
code += 'C Write only -\n'
code += '     8     yield, dyieldDtemp, dyieldDeqps,\n'
code += '     9     stateNew )\n'
code += 'C\n'
code += "      include 'vaba_param.inc'\n"
code += 'C\n'
code += '      dimension nElement(nblock),\n'
code += '     1     props(nprops),\n'
code += '     2     tempOld(nblock),\n'
code += '     3     fieldOld(nblock,nfieldv),\n'
code += '     4     stateOld(nblock,nstatev),\n'
code += '     5     tempNew(nblock),\n'
code += '     6     fieldNew(nblock,nfieldv),\n'
code += '     7     eqps(nblock),\n'
code += '     8     eqpsRate(nblock),\n'
code += '     9     yield(nblock),\n'
code += '     1     dyieldDtemp(nblock),\n'
code += '     2     dyieldDeqps(nblock,2),\n'
code += '     3     stateNew(nblock,nstatev)\n'
code += 'C\n'
code += '      character*80 cmname\n'
code += 'C\n'
code += '      do k = 1, nblock\n'

code += comment + 'xepsp = (eqps(k) - minEntries[0]) / rangeEntries[0]\n'
line = 'xepsp = (eqps(k)-' + var%(minEntries[0]) + ')/' + var%(rangeEntries[0])
code += toFortran(line)

code += comment + 'xdepsp = (log(depsp/logBase) - minEntries[1]) / rangeEntries[1]\n'
line = 'xdepsp = (log(eqpsRate(k)/' + var%(logBase[0]) + ')-' + var%(minEntries[1]) + ')/' + var%(rangeEntries[1])
code += toFortran(line)

code += comment + 'xtemp = (temp - minEntries[2]) / rangeEntries[2]\n'
line = 'xtemp = (tempNew(k)-' + var%(minEntries[2])+')/' + var%(rangeEntries[2])
code += toFortran(line)

code += comment + 'xa = np.exp(-w1.dot(x) - b1)\n'
for i in range(b1.shape[0]):
    line = 'xa' + str(i) + " = exp(-" + var%(w1[i,0]) + '*xepsp-' + var%(w1[i,1]) + '*xdepsp|-' + var%(w1[i,2]) + '*xtemp-' + var%(b1[i,0]) + ')'
    code += toFortran(line)

code += comment + 'xb = 1 + xa\n'
for i in range(b1.shape[0]):
    line = 'xb' + str(i) + " = 1.0 + xa"+str(i)
    code += toFortran(line)
    
code += comment + 'xc = w2.dot(xb) + b2\n'
for i in range(b2.shape[0]):
    line = 'xc' + str(i) + ' = '
    for j in range(w2.shape[1]):
        if (j!=0): line += '|+'
        line += var%(w2[i,j]) + '/xb' + str(j)
    line += "|+" + var%(b2[i,0])
    code += toFortran(line)
    
code += comment + 'xd = exp(-xc)\n'
for i in range(b2.shape[0]):
    line = 'xd'+str(i) + ' = exp(-xc' + str(i) + ')'
    code += toFortran(line)
    
code += comment + 'xsig = w3.dot(xd) + b3\n'
line = 'xsig = '
for j in range(w3.shape[1]):
    if (j!=0): line += '|+'
    line += var%(w3[0,j]) + '/(1.0 + xd'+str(j)+')'
line+="|+" + var%(b3[0,0])
code += toFortran(line)
    
code += comment + 'ya = w3v*(xd / (1 + xd)**2)\n'
for i in range(b2.shape[0]):
    line = 'ya' + str(i) + ' = ' + var%(w3[0,i]) + '*(xd'+str(i) + '/(1.0+xd' + str(i) + ')**2)'
    code += toFortran(line)

code += comment + 'yb = xa / (1 + xa)**2\n'
for i in range(b1.shape[0]):
    line = 'yb' + str(i) + ' = xa'+str(i) + ' / xb'+str(i) + '**2'
    code += toFortran(line)

code += comment + 'yc = (w2.T).dot(ya) * xd\n'
for i in range(b1.shape[0]):
    line = 'yc' + str(i) + ' = ('
    for j in range(b2.shape[0]):
        if(j>0): line += '|+'
        line += var%(w2[j,i]) + '*ya' + str(j)
    line += ')*yb' + str(i)
    code += toFortran(line)
    
code += comment + 'yd = (w1.T).dot(yc)\n'
for i in range(w1.shape[1]):
    line = 'yd' + str(i) + ' = '
    for j in range(w1.shape[0]):
        if (j>0): line += '|+'
        line += var%(w1[j,i]) + '*yc' + str(j)
    code += toFortran(line)

line = 'Yield(k) = ' + var%(rangeEntries[3]) + '*xsig+' + var%(minEntries[3])
code += toFortran(line)

line = 'dyieldDeqps(k,1) = ' + var%(rangeEntries[3]/rangeEntries[0]) + '*yd0'
code += toFortran(line)
line = 'dyieldDeqps(k,2) = ' + var%(rangeEntries[3]/rangeEntries[1]) + '*yd1 / eqpsRate(k)'
code += toFortran(line)
line = 'dyieldDtemp(k) = ' + var%(rangeEntries[3]/rangeEntries[2]) + '*yd2'
code += toFortran(line)

code += '      end do\n'
code += 'C\n'
code += '      return\n'
code += '      end\n'

In [88]:
text_file = open("VUHARD-ANN.f", "w")
text_file.write(code)
text_file.close()

In [15]:
testData[0,:]


array([ 6.49997854e-01,  1.79894090e+04,  3.35018578e+02,  1.23210657e+03,
        1.32070915e+02,  5.60676950e-04, -9.25727265e-01])

In [16]:
testData[0]
epsp = testData[0,0]
depsp = testData[0,1]
temp = testData[0,2]
epsp,depsp,temp

(0.6499978537536213, 17989.408960982073, 335.01857843791004)

In [18]:
yieldStress, testData[0,3]

(1232.228229196241, 1232.1065669321008)

## Dérivation directe

In [19]:
def tanhPrime2Layers(x):
    w3v = w3.reshape(w3.shape[1],1)
    tanhx = np.tanh(w1.dot(x) + b1)
    p2 = w3v * (1 - np.tanh(w2.dot(tanhx) + b2)**2)
    p3 = (w2.T).dot(p2)
    p5 = p3 * (1 - tanhx**2)
    s = (w1.T).dot(p5)
    return s

In [20]:
def sigmoidPrime2Layers(x):
    w3v = w3.reshape(w3.shape[1],1)
    expx = np.exp(-(w1.dot(x) + b1))
    exp2 = np.exp(w2.dot(1/(1 + expx)) + b2)
    p1 = w3v*(exp2 / (1 + exp2)**2)
    p2 = expx / (1 + expx)**2
    s = (w1.T).dot((w2.T).dot(p1) * p2)
    return s

In [24]:
derYieldStress1,derYieldStress2,derYieldStress3

(132.0722278659824, 0.0005599218637349831, -0.9256411664152612)

In [25]:
x=np.array([[xepsp],[xdepsp],[xtemp]])
x

array([[0.64999785],
       [0.90552119],
       [0.65628871]])

In [26]:
if ('tanh' in ANN_name) : s = tanhPrime2Layers(x)
elif ('sigmoid' in ANN_name) : s = sigmoidPrime2Layers(x)
else : s = 0
s

array([[ 0.13510455],
       [ 0.11148621],
       [-0.45450889]])

In [27]:
scaleOut = np.array([[rangeEntries[3]/rangeEntries[0]],[rangeEntries[3]/rangeEntries[1]],[rangeEntries[3]/rangeEntries[2]]])
scaleOut

array([[977.55571504],
       [ 90.34895996],
       [  2.03657441]])

In [28]:
scaled = s*scaleOut
scaled

array([[132.07222787],
       [ 10.07266339],
       [ -0.92564117]])

In [29]:
sigEpsNN = deps
sigEps = scaled[0][0]
print(sigEpsNN)
print('Ana : ',sigEps, abs((sigEpsNN-sigEps)/sigEps))

NameError: name 'deps' is not defined

In [ ]:
scaled[1][0]/testData[0,1]

In [ ]:
sigEpspNN = scaled[1][0]/testData[0,1]
print(sigEpspNN)
print('Num : ',sigEpspN, abs((sigEpspNN-sigEpspN)/sigEpspN))
print('Ana : ',sigEpsp, abs((sigEpspNN-sigEpsp)/sigEpsp))

In [ ]:
sigTNN = scaled[2][0]
print(sigTNN)
print('Num : ',sigTN, abs((sigTNN-sigTN)/sigTN))
print('Ana : ',sigT, abs((sigTNN-sigT)/sigT))